In [1]:
import pandas as pd
from threading import Thread
pd.set_option('max_colwidth',500)
import multiprocessing

In [5]:
import sys
sys.path.append("../scripts/")
import clean_text
from importlib import reload
reload(clean_text)
clean_text.clean('@esto es un #tweet. Hola!!😀')

'@ esto es un # tweet hola ! 😀'

In [4]:
tweets = pd.read_csv('../datasets/tweets_202007282056.csv')

In [7]:
df = tweets.copy()
df.index = df.tweet_id.values
print(len(df))
print(df.columns)
df.head()

2240646
Index(['tweet_id', 'conversation_id', 'user_id', 'created_at', 'lang',
       'retweet_count', 'favorite_count', 'reply_count', 'quote_count',
       'is_reply', 'replied_id', 'replied_user_id', 'is_quote', 'quoted_id',
       'full_text', 'source', 'scrapped_at', 'user_mentions_count',
       'hashtags_count', 'urls_count', 'media_count', 'replied_original'],
      dtype='object')


,tweet_id,conversation_id,user_id,created_at,lang,retweet_count,favorite_count,reply_count,quote_count,is_reply,...,is_quote,quoted_id,full_text,source,scrapped_at,user_mentions_count,hashtags_count,urls_count,media_count,replied_original
1000001051574046720,1000001051574046720,1000001051574046720,323956672,2018-05-25 13:10:04.0,es,9,12,1,0,0,...,0,NaN,Feliz día de la Patria! 🇦🇷 https://t.co/XfrTi29VcC,Twitter for iPhone,2020-07-28 05:10:54.0,0,0,0,1,NaN
1000002592955609088,1000002592955609088,1000002592955609088,53851352,2018-05-25 13:16:11.0,es,596,1486,2,4,0,...,0,NaN,"No busquen explicación, es amor.\n#117AñosDeHistoria https://t.co/b2lueUKodj",Twitter for Android,2020-07-28 04:26:30.0,0,1,0,1,NaN
1000002668474101761,1000002668474101761,1000002668474101761,69668594,2018-05-25 13:16:29.0,es,5,23,0,0,0,...,0,NaN,🇦🇷 La Patria somos todos y la construimos juntos. #FelizDíaDeLaPatria #25deMayo https://t.co/8IK54CBtoW,Twitter for Android,2020-07-28 02:21:15.0,0,2,0,1,NaN
1000003177448697857,1000003177448697857,1000003177448697857,834702151,2018-05-25 13:18:31.0,es,9897,18496,141,691,0,...,0,NaN,- Sos friolenta?\n- https://t.co/qkQNmQTwxE,TweetDeck,2020-07-28 12:31:42.0,0,0,0,1,NaN
1000003312597585921,1000003312597585921,1000003312597585921,971923363038973952,2018-05-25 13:19:03.0,es,1,7,0,0,0,...,0,NaN,Feliz Dia de la Patria! ❤ https://t.co/dqSyZOLt4p,Twitter for Android,2020-07-28 17:33:35.0,0,0,0,1,NaN


In [9]:
print(len(df[(df.favorite_count >= 5) & (df.lang == 'es')]))
print(len(df[df.lang=='es']))
df.sample(1)[['tweet_id', 'full_text']].values[0]

1118383
2058360


array([1243263397308874756,
       'La flexibilidad laboral de USA a pleno. Asi como se destruye empleo en el instante que se da la crisis, se genera cuando la economía da vuelta. Esta es la mejor forma de generar empleo de calidad a largo plazo..., no dar subsidios prohibir despidos... https://t.co/sXJVYM5oGF'],
      dtype=object)

In [10]:
from joblib import Parallel, delayed

def paralell_func(values, func, frac=8):
    r = Parallel(n_jobs=8, backend='threading')(delayed(func)(i) for i in values)
    return r

def clean_fn(row):
    row[1] = clean_text.clean(row[1])
    return row

cleaned_texts = paralell_func(df.sample(5)[['tweet_id', 'full_text']].values, clean_fn)

In [13]:
list(zip(*cleaned_texts))

[(1026963136786493440,
  1287795529875951618,
  1171747810586386432,
  1232126096260378625,
  1285878209054113792),
 ('entre los vicios más graves de cualquier acto administrativo y que produce la nulidad a futuro es la falta de fundamentación debemos extremar estos recaudos si no queremos ser parte de las irregularidades de un miembro del poder judicial advierte @ pilattivergara',
  '# embiarg riesgo país 🇦 🇷 2197 2020 07 27 variación diaria 1 04 URL',
  'solo los farmacéuticos entienden lo que dice ahí',
  'con el agua hasta el ogt pero cargando los hijos para saquearnos cada dia mas son una manada de orcos vagos y lisergicos pero manzurok los cuida gente creo que a mi marido lo llamaran cr venga a buscar a su sra cra de la comisaria 😂',
  '# ahora diplomáticos chinos queman documentos luego de que eeuu ordenara el cierre del consulado de china en houston URL')]

In [14]:
frac = multiprocessing.cpu_count()
values = df[['tweet_id', 'full_text']].values
cleaned_texts = paralell_func(values, clean_fn, frac)

In [15]:
to_insert = list(zip(*cleaned_texts))
df.loc[to_insert[0], 'full_text'] = to_insert[1]

In [16]:
df.head(20)[['full_text']]

,full_text
1000001051574046720,feliz día de la patria ! 🇦 🇷 URL
1000002592955609088,no busquen explicación es amor # 117añosdehistoria URL
1000002668474101761,🇦 🇷 la patria somos todos y la construimos juntos # felizdíadelapatria # 25demayo URL
1000003177448697857,sos friolenta ? URL
1000003312597585921,feliz dia de la patria ! URL
1000003529396883456,participando del tedeum por el # 25demayo en la parroquia santa maría del carmen # necochea URL
1000004585124253699,feliz día de la patria ! el día que comenzamos a ser libres URL
1000004674483900418,muchas gracias a todos los vecinos que me acompañaron en el tradicional toque de diana comenzamos con los festejos en mendoza de los 208 años de la revolución # 25demayo URL
1000005724670488577,# ahora hablamos en vivo por blanco sobre negro por @ radioprovchaco fm 101 5
1000005727933657089,# exclusiva ahora hablamos con el actor @ cesarbordonok que interpreta a # hugolópez el manager de luis miguel en la serie de # netflix # luismiguellaserie URL


In [17]:
# df.to_pickle('../datasets/tweets_cleaned.pkl')

In [7]:
df = pd.read_pickle('../datasets/tweets_cleaned.pkl')

In [21]:
len(df[(df.favorite_count >= 2) & (df.lang == 'es')])

1418545

In [22]:
corpus_to_save = df[(df.lang == 'es')].full_text.values
with open("../datasets/tweets_corpus.txt", "w", encoding='utf-8') as text_file:
    text_file.write(
        "\n".join(corpus_to_save)
    )